## Before you start:
**Tools → Settings → Editor → completions / suggestions / linting → disable**

**Task 1:** Study how basic reference counting works
Create a string and investigate its reference count at different stages. Return a tuple: (count_after_creation, count_after_ref, count_after_del)

In [15]:
def task1():
    import sys

    s = "string"
    count_after_creation = sys.getrefcount(s)

    ref = s
    count_after_ref = sys.getrefcount(s)

    del ref
    count_after_del = sys.getrefcount(s)

    return (count_after_creation, count_after_ref, count_after_del)

print( task1())


(771, 772, 771)


In [14]:
import sys
import gc
import weakref

**Task 2:** Function impact on Reference Counting
Create a list and pass it to a function.Compare reference count before and after function call.Return a tuple: (count_before_call, count_during_call, count_after_call)

In [ ]:
def task2():

      def process_list(lst):
          return sys.getrefcount(lst)

      my_list = [1, 2, 3, 4, 5]

      count_before_call = sys.getrefcount(my_list)

      count_during_call = process_list(my_list)

      count_after_call = sys.getrefcount(my_list)

      return (count_before_call, count_during_call, count_after_call)

print( task2())

Task 2: (2, 4, 2)


**Task 3:** Cyclic references and Memory Leaks
Create two objects with a cyclic reference, then break it.Use gc to check the number of collected objects.Return the number of objects collected by gc after breaking the reference.

In [19]:
def task3():
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None

    node1 = Node("Node1")
    node2 = Node("Node2")

    node1.ref = node2
    node2.ref = node1

    del node1
    del node2

    collected = gc.collect()

    return collected

print(task3())

10


**Task 4:** Comparing Reference Count for different data types
Compare reference count for numbers, strings, lists and dictionaries.Return a dictionary with reference count for each type after creation.

In [ ]:
def task4():
    num = 42
    text = "example"
    lst = [1, 2, 3]
    dct = {'key': 'value'}

    result = {
        'int': sys.getrefcount(num),
        'str': sys.getrefcount(text),
        'list': sys.getrefcount(lst),
        'dict': sys.getrefcount(dct)
    }

    return result

print(task4())

Task 4: {'int': 37, 'str': 5, 'list': 2, 'dict': 2}


**Task 5:** Weak References
Create two objects with a weak reference between them.Ensure the objects can be deleted by the garbage collector.Return True if the weakref does not increase reference count.

In [ ]:
def task5():
    class Data:
        def __init__(self, value):
            self.value = value

    obj = Data(100)

    refcount_before = sys.getrefcount(obj)

    weak_obj = weakref.ref(obj)

    refcount_after = sys.getrefcount(obj)

    return refcount_before == refcount_after

print("Task 5:", task5())

Task 5: True


**Task 6:** Monitoring the Garbage Collector
Register a callback function to track GC activity.Return a list of events tracked by the callback.

In [ ]:
def task6():
    events = []

    def callback(phase, info):
        events.append({'phase': phase, 'info': info})

    gc.callbacks.append(callback)

    for _ in range(100):
        a = []
        b = []
        a.append(b)
        b.append(a)

    gc.collect()

    gc.callbacks.remove(callback)

    return events

print( task6())

Task 6: [{'phase': 'start', 'info': {'generation': 2, 'collected': 0, 'uncollectable': 0}}, {'phase': 'stop', 'info': {'generation': 2, 'collected': 218, 'uncollectable': 0}}]


**Task 7:** GC Generation Analysis
Create objects and trace their movement between GC generations.Return a tuple with the number of objects in each generation before and after object creation.

In [33]:
def task7():
    before = tuple(len(gc.get_objects(generation=i)) for i in range(3))

    objects = []
    for i in range(1000):
        obj = {'data': i}
        objects.append(obj)

    after = tuple(len(gc.get_objects(generation=i)) for i in range(3))

    return (before, after)

print( task7())

((296, 13, 73586), (3, 296, 73586))


**Task 8:** Monitoring Garbage Collection Thresholds
Study how GC generation counters change when creating objects.Return a dictionary with the state of the counters before and after object creation.

In [ ]:
def task8():
    initial_threshold = gc.get_threshold()
    initial_count = gc.get_count()

    objects = []
    for i in range(1000):
        objects.append({'index': i})

    after_creation_count = gc.get_count()

    collected = gc.collect(0)
    after_collect_count = gc.get_count()

    return {
        'thresholds': initial_threshold,
        'initial_count': initial_count,
        'after_creation': after_creation_count,
        'after_collect_gen0': after_collect_count,
        'collected_objects': collected
    }

print(task8())

Task 8: {'thresholds': (700, 10, 10), 'initial_count': (162, 9, 0), 'after_creation': (445, 10, 0), 'after_collect_gen0': (0, 11, 0), 'collected_objects': 0}


**Task 9:** Quick GC check
Check if the GC collects cyclic references.Return the difference in the number of objects before and after collection.

In [ ]:
def task9():
    a, b, c = [], [], []
    a.append(b)
    b.append(c)
    c.append(a)

    del a, b, c

    count_before = len(gc.get_objects())
    gc.collect()
    count_after = len(gc.get_objects())

    return count_before - count_after

print(task9())

2


In [36]:
def task9_solution():
    # Quickly create circular references
    a, b, c = [], [], []
    a.append(b); b.append(c); c.append(a)  # Cycle a->b->c->a

    count_before = len(gc.get_objects())
    collected = gc.collect()
    count_after = len(gc.get_objects())

    return count_before - count_after, collected

print("Task 9:", task9_solution())

Task 9: (-18, 0)


**Task 10:** Detector for "undying" objects
Find objects that survive forced garbage collection.Output the number of such objects.

In [37]:
def task10():
    gc.collect()

    before = set(id(obj) for obj in gc.get_objects())

    a = []
    b = []
    a.append(b)
    b.append(a)

    del a, b

    gc.collect()

    after = set(id(obj) for obj in gc.get_objects())

    surviving = len(after & before)

    return surviving

print("Task 10:", task10())

Task 10: 73907
